In [1]:
# Import packages
import numpy as np
import pandas as pd
import scipy.stats as sts
import os
import matplotlib.pyplot as plt
import scipy.optimize as opt
import warnings
warnings.filterwarnings('ignore')
from scipy import special
from scipy.integrate import quad

In [2]:
data = np.loadtxt('MacroSeries.txt', delimiter=",")
c = data[:, 0]
k = data[:, 1]
w = data[:, 2]
r = data[:, 3]
y = data[:, 4]

In [3]:
weight = np.identity(6)
alpha_init = 0.8
beta = 0.99
rho_init = 0.7
mu_init = 0.2
sigma_init = 0.2
params_init = (alpha_init, rho_init, mu_init, sigma_init)
bnds = ((0.01, 0.99), (-0.99, 0.99), (5, 14), (0.01, 1.1))
args = (c, k, w, r, y, weight)

In [4]:
def data_moments(c, k, y):
    mean_c = c.mean()
    mean_k = k.mean()
    cy=c/y
    mean_cy = cy.mean()
    var_y = y.var()
    corr_ck = np.corrcoef(c, k)[0, 1]
    corr_c = np.corrcoef(c[1:], c[:-1])[0, 1]
    return mean_c, mean_k, mean_cy, var_y, corr_ck, corr_c


In [5]:
meank = k.mean()
def simulate_moments(alpha, rho, mu, sigma, T=100, S=1000):
    z1 = mu
    k1 = meank
    epsilon = np.random.normal(0, scale=sigma, size=(S, T)) 
    zmat = np.zeros((S, T))
    kmat = np.zeros((S, T))
    cmat = np.zeros((S, T))
    ymat = np.zeros((S, T))
    zmat[:, 0] = z1 * np.ones(S)
    kmat[:, 0] = k1 * np.ones(S)
    
    for i in range(1, T):
        zmat[:, i] = rho * zmat[:, i-1] + (1 - rho) * mu + epsilon[:, i]
        kmat[:, i] = alpha * beta * np.exp(zmat[:, i]) + kmat[:, i-1] ** alpha
    wmat = (1 - alpha) * np.exp(zmat) * kmat ** alpha
    rmat = alpha * np.exp(zmat) * kmat ** (alpha - 1)
    ymat = np.exp(zmat)*pow(kmat,alpha)
    cmat[:, 0] = wmat[:, 0] + rmat[:, 0] * kmat[:, 0]
    for i in range(1, T):
        cmat[:, i] = wmat[:, i] + rmat[:, i] * kmat[:, i] - kmat[:, i-1]
   
    
   
    smean_c = cmat.mean(axis=1).mean()
    smean_k = kmat.mean(axis=1).mean()
    svar_y = ymat.var(axis=1).mean()
    cy = cmat/ymat
    smean_cy = cy.mean(axis=1).mean()
    corr_ck = np.zeros(T)
    corr_c = np.zeros(T - 1)
    
    for i in range(T):
        corr_ck[i] = np.corrcoef(cmat[:, i], kmat[:, i])[0, 1]
    for i in range(T-1):
        corr_c[i] = np.corrcoef(cmat[1:, i], cmat[:-1, i])[0, 1]  
    scorr_ck = corr_ck[2:].mean()
    scorr_c = corr_c[1:].mean()
    
    return smean_c, smean_k, svar_y, smean_cy, scorr_ck, scorr_c


In [6]:
def err_vec(c, k, w, r, y, alpha, rho, mu, sigma):
    mo_moments = np.array(simulate_moments(alpha, rho, mu, sigma))
    da_moments = np.array(data_moments(c, k, y))
    
    err_vec = ((mo_moments - da_moments) / da_moments)
    return err_vec

def crit(params, *args):
    alpha, rho, mu, sigma = params
    c, k, w, r, y, weight  = args
    err = err_vec(c, k, w, r, y, alpha, rho, mu, sigma)
    crit_val = np.dot(np.dot(err.T, weight), err) 
    
    return crit_val

In [7]:
results = opt.minimize(crit, params_init, args=(args), bounds = bnds, method = 'L-BFGS-B')
print(results)

      fun: 1.5092237317742576e+26
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([1.70620634e+30, 2.98308623e+29, 1.28827832e+30, 1.87792770e+30])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 145
      nit: 4
   status: 0
  success: True
        x: array([0.77072647, 0.71079819, 5.        , 0.19296242])
